In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, models
import numpy
import matplotlib
import os
import time
from torchsummary import summary
import copy

In [6]:
# 定义数据变换的函数
data_transform = {'train': transforms.Compose([
                transforms.RandomSizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
                'val': transforms.Compose(
                    [transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
                }
folder_path = "./data/beef"
image_datasets = {x: datasets.ImageFolder(os.path.join(folder_path, x), data_transform[x]) for x in ['train','val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                batch_size=4,
                                                shuffle=True,
                                                num_workers=4)
                        for x in ['train','val']}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataset_size = {x: len(image_datasets[x]) for x in ['train', 'val']}
                                             

In [7]:
# 创建模型
# 加载预训练的模型
model = models.resnet18(pretrained=True)
# 查看模型的状态
# print(model)
summary(model,(3,244,244))

# 获得模型中fc 层
model_fc_features = model.fc.in_features
print(model.fc.out_features)
print(model_fc_features)
model.fc = nn.Linear(model_fc_features,2)

print(model.fc.out_features)
model = model.to(device)
criterion = nn.CrossEntropyLoss()

optim = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

lr_scheduler = lr_scheduler.StepLR(optim, step_size=7, gamma=0.1)




----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]          36,864
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
       BasicBlock-11           [-1, 64, 61, 61]               0
           Conv2d-12           [-1, 64, 61, 61]          36,864
      BatchNorm2d-13           [-1, 64, 61, 61]             128
             ReLU-14           [-1, 64,

AttributeError: 'SGD' object has no attribute 'SGD'

In [8]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=15):
    since = time.time()

    #获得模型的参数
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epock{epoch}/{num_epochs - 1}")
        print("-" * 20)

        for phase in ['train','val']:
            # 设置模型是是训练还是验证阶段
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                # 加载数据，并将数据放入cpu或者GPU中
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 对梯度清零
                optimizer.zero_grad()

                #是否做梯度运算
                with torch.set_grad_enabled(phase == 'train'):
                    # 前向传播
                    outputs = model(inputs)
                    # 获得推理的结果，这里是logistic
                    _, preds = torch.max(outputs, 1)
                    # 计算损失值
                    loss = criterion(outputs, labels)

                    # 在训练阶段需要反向传播，并更新梯度值
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 计算一个batch的损失值和准确率
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)  

            # 更新学习率
            if phase == 'train':
                scheduler.step()

            # 计算损失值和准确率
            epoch_loss = running_loss / dataset_size[phase] 
            epoch_acc = running_corrects.double() / dataset_size[phase] 

            print('{} Loos: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc
            ))  

            # 将最好的模型进行保存
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))  

    print('Best val Acc: {:4f}'.format(best_acc))

    # 加载模型
    model.load_state_dict(best_model_wts)
    return model          


In [9]:
model_ft = train_model(model,criterion,optim,lr_scheduler, num_epochs=25)

Epock0/24
--------------------
train Loos: 0.7162 Acc: 0.5082
val Loos: 0.7179 Acc: 0.5229
Epock1/24
--------------------
train Loos: 0.7112 Acc: 0.5492
val Loos: 0.7000 Acc: 0.5294
Epock2/24
--------------------
train Loos: 0.7082 Acc: 0.4918
val Loos: 0.7173 Acc: 0.5229
Epock3/24
--------------------
train Loos: 0.7004 Acc: 0.5369
val Loos: 0.6989 Acc: 0.5294
Epock4/24
--------------------
train Loos: 0.7096 Acc: 0.5451
val Loos: 0.7019 Acc: 0.5229
Epock5/24
--------------------
train Loos: 0.6875 Acc: 0.5451
val Loos: 0.7103 Acc: 0.5425
Epock6/24
--------------------
train Loos: 0.7056 Acc: 0.5328
val Loos: 0.7110 Acc: 0.5033
Epock7/24
--------------------
train Loos: 0.6874 Acc: 0.5451
val Loos: 0.7195 Acc: 0.5294
Epock8/24
--------------------
train Loos: 0.6931 Acc: 0.5451
val Loos: 0.7233 Acc: 0.5229
Epock9/24
--------------------
train Loos: 0.7204 Acc: 0.4959
val Loos: 0.7179 Acc: 0.4967
Epock10/24
--------------------
train Loos: 0.6940 Acc: 0.5697
val Loos: 0.7240 Acc: 0.509

AttributeError: 'function' object has no attribute 'copy'